In [1]:
import json, urllib.request, sys, math, re
import numpy as np

graph = {}
paths = []

In [2]:
def main():
    forex_rates = get_rates()
    print("Current Forex Rates:")
    print(forex_rates)
    print("\n")
    gr = get_graph(forex_rates)
    
    for key in graph:
        path = bellman_ford(graph, key)
        if path not in paths and not None:
            paths.append(path)

    for path in paths:
        if path == None:
            print("No Arbitrage opportunity detected in current rates :(")
        else:
            money = 100
            print("<---Arbitrage cycle detected--->")
            print("Starting with %(money)i in %(currency)s" % {"money":money,"currency":path[0]})

            for i,value in enumerate(path):
                if i+1 < len(path):
                    start = path[i]
                    end = path[i+1]
                    rate = math.exp(-graph[start][end])
                    money *= rate
                    print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start":start,"end":end,"rate":rate,"money":money})
        print("\n")

In [3]:
def get_rates():
    try:
        forex_url = urllib.request.urlopen("http://fx.priceonomics.com/v1/rates/")
        forex_r = forex_url.read()
        rates = json.loads(forex_r)
    except Exception as e:
        print >>sys.stderr, "Error getting rates:", e
        sys.exit(1)
    
    return rates

In [4]:
def get_graph(forex_rates):
    pattern = re.compile("([A-Z]{3})_([A-Z]{3})")
    for key in forex_rates:
        matches = pattern.match(key)
        conversion_rate = -math.log(float(forex_rates[key]))
        from_rate = matches.group(1)
        to_rate = matches.group(2)
        if from_rate != to_rate:
            if from_rate not in graph:
                graph[from_rate] = {}
            graph[from_rate][to_rate] = float(conversion_rate)
    return graph

In [5]:
def initialize(graph, source):
    d = {} # Stands for destination
    p = {} # Stands for predecessor
    for node in graph:
        d[node] = float('Inf') # We start admiting that the rest of nodes are very very far
        p[node] = None
    d[source] = 0 # For the source we know how to reach
    return d, p

In [6]:
def relax(node, neighbor, graph, d, p):
    # If the distance between the node and the neighbor is lower than the one I have now
    if d[neighbor] > d[node] + graph[node][neighbor]:
        # Record this lower distance
        d[neighbor]  = d[node] + graph[node][neighbor]
        p[neighbor] = node

In [7]:
def retrace_negative_loop(p, start):
    arbitrageLoop = [start]
    next_node = start
    while True:
        next_node = p[next_node]
        if next_node not in arbitrageLoop:
            arbitrageLoop.append(next_node)
        else:
            arbitrageLoop.append(next_node)
            arbitrageLoop = arbitrageLoop[arbitrageLoop.index(next_node):]
            return arbitrageLoop


In [8]:
def bellman_ford(graph, source):
    d, p = initialize(graph, source)
    for i in range(len(graph)-1): #Run this until is converges
        for u in graph:
            for v in graph[u]: #For each neighbor of u
                relax(u, v, graph, d, p) #Lets relax it


    # Step 3: check for negative-weight cycles
    for u in graph:
        for v in graph[u]:
            if d[v] < d[u] + graph[u][v]:
                return(retrace_negative_loop(p, source))
    return None

In [9]:
if __name__ == '__main__':
    main()

Current Forex Rates:
{'USD_JPY': '89.2384542', 'USD_USD': '1.0000000', 'JPY_EUR': '0.0085665', 'BTC_USD': '113.7636475', 'JPY_BTC': '0.0000915', 'USD_EUR': '0.6775256', 'EUR_USD': '1.3559767', 'EUR_JPY': '138.9307164', 'JPY_USD': '0.0110122', 'BTC_BTC': '1.0000000', 'EUR_BTC': '0.0118718', 'BTC_JPY': '11645.5539436', 'JPY_JPY': '1.0000000', 'BTC_EUR': '83.9149548', 'EUR_EUR': '1.0000000', 'USD_BTC': '0.0072307'}


<---Arbitrage cycle detected--->
Starting with 100 in EUR
EUR to JPY at 138.930716 = 13893.071640
JPY to EUR at 0.008566 = 119.014998


<---Arbitrage cycle detected--->
Starting with 100 in JPY
JPY to EUR at 0.008566 = 0.856650
EUR to JPY at 138.930716 = 119.014998


